
# 🌱 ReLeaf – BKF × WFO Mapping (Structured Outputs)

รายงานนี้รวม **โค้ดทำความสะอาด/แมป WFO** ฉบับปรับปรุงให้ **บันทึกผลลัพธ์อย่างเป็นระเบียบ** ลงในโฟลเดอร์ `Processed data` 
พร้อม Markdown อธิบายขั้นตอนสำคัญเพื่อให้ง่ายต่อการตรวจสอบย้อนกลับและออกรายงาน



## 🗂️ โครงสร้างไฟล์ผลลัพธ์ (ภายใต้ `Processed data/`)

```
Processed data/
├─ 00_logs/                # บันทึก/ข้อความ log เพิ่มเติม (ถ้ามี)
├─ 01_enriched/            # ชุดข้อมูลหลักหลัง enrich/mapping
├─ 02_audit/               # ไฟล์ตรวจสอบ/ขัดแย้ง (conflicts & diagnostics)
├─ 03_summary/             # ตัวชี้วัดสรุป/metrics (JSON/CSV)
└─ runs/YYYYMMDD_HHMM/     # สำเนา snapshot ของผลลัพธ์ทุกไฟล์ตามเวลาที่รัน
```


In [ ]:
# -*- coding: utf-8 -*-
# ====== Imports ======
import pandas as pd
import numpy as np
from pathlib import Path
import re, json, shutil
from collections import Counter
from datetime import datetime

# ====== CONFIG ======
# Root project folder
ROOT_DIR = Path("/Users/thanwaratkeratipasuwat/Desktop/dsi314")

# Input (Raw data)
RAW_DIR  = ROOT_DIR / "Raw data"
BKF_PATH  = RAW_DIR / "bkf_eflora_species_dedup.csv"   # BKF หลัง dedup
OUT7_PATH = RAW_DIR / "output (7).csv"                 # จาก WFO map tools
CAND_PATH = RAW_DIR / "candidates (3).csv"             # candidates เพิ่มเติม

# Outputs base
OUT_DIR   = ROOT_DIR / "Processed data"

# Organized output dirs
LOG_DIR     = OUT_DIR / "00_logs"
ENRICH_DIR  = OUT_DIR / "01_enriched"
AUDIT_DIR   = OUT_DIR / "02_audit"
SUMMARY_DIR = OUT_DIR / "03_summary"
RUN_STAMP   = datetime.now().strftime("%Y%m%d_%H%M")
RUN_DIR     = OUT_DIR / "runs" / RUN_STAMP

for p in [OUT_DIR, LOG_DIR, ENRICH_DIR, AUDIT_DIR, SUMMARY_DIR, RUN_DIR]:
    p.mkdir(parents=True, exist_ok=True)

print("📁 Paths Configuration:")
print(f"BKF_PATH         : {BKF_PATH}")
print(f"OUT7_PATH        : {OUT7_PATH}")
print(f"CAND_PATH        : {CAND_PATH}")
print(f"OUT_DIR          : {OUT_DIR}")
print(f"ENRICH_DIR       : {ENRICH_DIR}")
print(f"AUDIT_DIR        : {AUDIT_DIR}")
print(f"SUMMARY_DIR      : {SUMMARY_DIR}")
print(f"RUN_DIR          : {RUN_DIR}")

# ====== HELPERS ======
def norm(x):
    if pd.isna(x): return ""
    return str(x).replace("\u00A0", " ").strip().lower()

def find_col(cols, candidates):
    """หาคอลัมน์แบบทนทานต่อเคสตัวพิมพ์"""
    cols_list = list(cols)
    for c in candidates:
        if c in cols_list:
            return c
    lower_map = {c.lower(): c for c in cols_list}
    for c in candidates:
        if c.lower() in lower_map:
            return lower_map[c.lower()]
    return None

def nonempty_series_like(df, col):
    """คืน Series ของคอลัมน์ (ถ้าไม่มีให้คืนซีรีส์ว่าง) และแปลง nan/None เป็น '' """
    if col is None or col not in df.columns:
        return pd.Series("", index=df.index)
    return (df[col].astype(str)
                 .str.strip()
                 .replace({"nan": "", "None": "", "NaT": ""}))

def wfo_clean(x):
    """ทำความสะอาด WFO -> รูปแบบ wfo-xxxx (ตัวพิมพ์เล็ก, ตัดอักขระแปลก)"""
    if pd.isna(x): return ""
    s = str(x).strip()
    if s == "" or s.lower() in {"none","nan"}:
        return ""
    s2 = s.lower()
    if not s2.startswith("wfo-"):
        s2 = "wfo-" + s2
    s2 = re.sub(r"[^a-z0-9\-]", "", s2)
    return s2

def prefer(left, right):
    """เลือก left ถ้าไม่ว่าง มิฉะนั้นใช้ right"""
    ls = left.astype(str).str.strip()
    rs = right.astype(str).str.strip()
    return np.where(ls != "", ls, rs)

def resolve_eff(df, colname, suffix):
    """คืนชื่อคอลัมน์ที่มีอยู่จริงหลัง merge (เช็คว่ามีการเติม suffix หรือไม่)"""
    if not colname:
        return None
    if f"{colname}{suffix}" in df.columns:
        return f"{colname}{suffix}"
    return colname if colname in df.columns else None

def _safe_save_csv(df, path):
    try:
        df.to_csv(path, index=False)
        print(f"[OK] Saved: {path}")
    except Exception as e:
        print(f"[ERR] Save failed: {path} -> {e}")

# ====== LOAD ======
df_bkf = pd.read_csv(BKF_PATH)
df_o7  = pd.read_csv(OUT7_PATH)
df_c   = pd.read_csv(CAND_PATH)

# ====== IDENTIFY KEY COLUMNS ======
bkf_spec = find_col(df_bkf.columns, ["specific_name"])
bkf_wfo  = find_col(df_bkf.columns, ["wfo_id"])
bkf_url  = find_col(df_bkf.columns, ["species_url"])
bkf_full = find_col(df_bkf.columns, ["wfo_full_name","wfo_name","full_name","accepted_name"])

if bkf_spec is None:
    raise KeyError("ไม่พบคอลัมน์ specific_name ใน BKF")

if bkf_wfo is None:
    df_bkf["wfo_id"] = np.nan
    bkf_wfo = "wfo_id"

o7_spec  = find_col(df_o7.columns, ["specific_name"])
o7_wfo   = find_col(df_o7.columns, ["wfo_id","wfo"])
o7_full  = find_col(df_o7.columns, ["wfo_full_name"])

c_spec   = find_col(df_c.columns, ["specific_name"])
c_wfo    = find_col(df_c.columns, ["wfo_id"])
c_full   = find_col(df_c.columns, ["wfo_full_name"])

# ====== NORMALIZE KEY & WFO ======
df_bkf["specific_name_clean"] = df_bkf[bkf_spec].apply(norm)
if bkf_url: df_bkf["species_url_clean"] = df_bkf[bkf_url].apply(norm)

if o7_spec: df_o7["specific_name_clean"] = df_o7[o7_spec].apply(norm)
if c_spec:  df_c["specific_name_clean"]  = df_c[c_spec].apply(norm)

df_bkf[bkf_wfo] = nonempty_series_like(df_bkf, bkf_wfo).apply(wfo_clean)
if o7_wfo:
    df_o7[o7_wfo] = nonempty_series_like(df_o7, o7_wfo).apply(wfo_clean)
if c_wfo:
    df_c[c_wfo]   = nonempty_series_like(df_c, c_wfo).apply(wfo_clean)

# ====== STEP 0: BASELINE ======
missing_before = df_bkf[bkf_wfo].eq("") | df_bkf[bkf_wfo].isna()
print(f"[BASELINE] Missing WFO (BKF): {int(missing_before.sum())} / {len(df_bkf)}")

# ====== STEP 1: JOIN output(7) ด้วย specific_name_clean ======
o7_use_cols = ["specific_name_clean"]
if o7_wfo:  o7_use_cols.append(o7_wfo)
if o7_full: o7_use_cols.append(o7_full)
df_o7_use = (df_o7[o7_use_cols].drop_duplicates("specific_name_clean")
             if o7_spec else pd.DataFrame(columns=o7_use_cols))

df1 = df_bkf.merge(df_o7_use, on="specific_name_clean", how="left", suffixes=("","_o7"))

# ====== STEP 2: PREP candidates — เก็บเฉพาะชื่อที่มี WFO เดียว ======
if (c_spec is not None) and (c_wfo is not None):
    cg = df_c.dropna(subset=[c_wfo]).copy()
    cg[c_wfo] = cg[c_wfo].astype(str).str.strip()
    uniq_one_key = (
        cg.groupby("specific_name_clean")[c_wfo].nunique()
          .reset_index(name="n").query("n == 1")[["specific_name_clean"]]
    )
    c_use = (
        cg.merge(uniq_one_key, on="specific_name_clean", how="inner")
          .drop_duplicates(subset=["specific_name_clean"])
    )
    c_use_cols = ["specific_name_clean", c_wfo]
    if c_full: c_use_cols.append(c_full)
    c_use = c_use[c_use_cols]
else:
    c_use = pd.DataFrame(columns=["specific_name_clean"])

df2 = df1.merge(c_use, on="specific_name_clean", how="left", suffixes=("","_cand"))

# ====== STEP 2.1: RESOLVE EFFECTIVE NAMES หลัง merge ======
o7_wfo_eff  = resolve_eff(df2, o7_wfo,  "_o7")
o7_full_eff = resolve_eff(df2, o7_full, "_o7")
c_wfo_eff   = resolve_eff(df2, c_wfo,   "_cand")
c_full_eff  = resolve_eff(df2, c_full,  "_cand")

# ====== STEP 3: เลือก wfo_id ตามลำดับความสำคัญ ======
bkf_val = nonempty_series_like(df2, bkf_wfo)
o7_val  = nonempty_series_like(df2, o7_wfo_eff) if o7_wfo_eff else pd.Series("", index=df2.index)
c_val   = nonempty_series_like(df2, c_wfo_eff)  if c_wfo_eff  else pd.Series("", index=df2.index)

bkf_has = bkf_val != ""
o7_has  = o7_val  != ""
c_has   = c_val   != ""

df2["wfo_id_final"] = np.select(
    [bkf_has, (~bkf_has) & o7_has, (~bkf_has) & (~o7_has) & c_has],
    [bkf_val, o7_val, c_val],
    default=""
)

df2["wfo_source"] = np.select(
    [bkf_has, (~bkf_has) & o7_has, (~bkf_has) & (~o7_has) & c_has],
    ["bkf_original","output7_exact","candidates_unique"],
    default="unfilled"
)

# ====== STEP 3.1: FLAG CONFLICTS ======
def diff_nonempty(a, b):
    a = a.astype(str).str.strip(); b = b.astype(str).str.strip()
    return (a != "") & (b != "") & (a != b)

df2["conflict_bkf_vs_o7"]   = diff_nonempty(bkf_val, o7_val)
df2["conflict_bkf_vs_cand"] = diff_nonempty(bkf_val, c_val)
df2["conflict_o7_vs_cand"]  = diff_nonempty(o7_val, c_val)
df2["any_conflict"] = df2[["conflict_bkf_vs_o7","conflict_bkf_vs_cand","conflict_o7_vs_cand"]].any(axis=1)

# ====== STEP 4: wfo_full_name จาก output(7) ======
if o7_wfo and o7_full:
    o7_id_name = (
        df_o7[[o7_wfo, o7_full]]
        .dropna(subset=[o7_wfo])
        .copy()
    )
    o7_id_name[o7_wfo]  = o7_id_name[o7_wfo].astype(str).str.strip()
    o7_id_name[o7_full] = o7_id_name[o7_full].astype(str).str.strip()
    o7_id_name = (
        o7_id_name[o7_id_name[o7_wfo] != ""]
        .drop_duplicates(subset=[o7_wfo], keep="first")
    )
    o7_map = dict(zip(o7_id_name[o7_wfo], o7_id_name[o7_full]))
    df2["wfo_full_name_final"] = df2["wfo_id_final"].map(o7_map).fillna("")
else:
    df2["wfo_full_name_final"] = ""

# ====== STEP 5: ผลลัพธ์มาตรฐาน ======
df_out = df2.copy()
df_out["wfo_id"] = df_out["wfo_id_final"].replace({"": np.nan})
df_out["wfo_full_name"] = df_out["wfo_full_name_final"].replace({"": np.nan})

# ทำความสะอาดคอลัมน์ชั่วคราว (เก็บเฉพาะคอลัมน์ผลลัพธ์หลัก)
drop_cols = []
for col in [
    o7_wfo_eff, o7_full_eff, c_wfo_eff, c_full_eff,
    "wfo_id_final", "wfo_full_name_final",
    "specific_name_clean", "species_url_clean",
    "conflict_bkf_vs_o7","conflict_bkf_vs_cand","conflict_o7_vs_cand","any_conflict"
]:
    if col and (col in df_out.columns) and (col not in ["wfo_id","wfo_full_name"]):
        drop_cols.append(col)
df_out = df_out.drop(columns=drop_cols, errors="ignore")

# ====== STEP 6: SAVE (Organized) ======
enriched_path = ENRICH_DIR / "bkf_wfo_enriched.csv"
unmatched = df_out[df_out["wfo_id"].isna() | (df_out["wfo_id"].astype(str).str.strip()=="")]
unmatched_path = ENRICH_DIR / "bkf_wfo_unmatched.csv"

_safe_save_csv(df_out, enriched_path)
_safe_save_csv(unmatched, unmatched_path)

# ====== STEP 7: AUDIT / DIAGNOSTIC ======
audit_cols = [bkf_spec, bkf_wfo, "specific_name_clean"]
if bkf_full: audit_cols.append(bkf_full)
if o7_wfo_eff:  audit_cols.append(o7_wfo_eff)
if o7_full_eff: audit_cols.append(o7_full_eff)
if c_wfo_eff:   audit_cols.append(c_wfo_eff)
if c_full_eff:  audit_cols.append(c_full_eff)
audit_cols = [c for i,c in enumerate(audit_cols) if c and (audit_cols.index(c) == i)]

df_audit = df2[audit_cols + [
    "wfo_id_final","wfo_source",
    "conflict_bkf_vs_o7","conflict_bkf_vs_cand","conflict_o7_vs_cand","any_conflict",
    "wfo_full_name_final"
]].copy()

rename_map = {}
if bkf_spec: rename_map[bkf_spec] = "bkf_specific_name"
if bkf_wfo:  rename_map[bkf_wfo]  = "bkf_wfo_id"
if o7_wfo_eff:  rename_map[o7_wfo_eff]  = "o7_wfo_id"
if c_wfo_eff:   rename_map[c_wfo_eff]   = "cand_wfo_id"
if bkf_full:    rename_map[bkf_full]    = "bkf_wfo_full_name"
if o7_full_eff: rename_map[o7_full_eff] = "o7_wfo_full_name"
if c_full_eff:  rename_map[c_full_eff]  = "cand_wfo_full_name"
rename_map["wfo_full_name_final"] = "wfo_full_name_final"

df_audit.rename(columns=rename_map, inplace=True)

audit_conflicts_path  = AUDIT_DIR / "bkf_wfo_audit_conflicts.csv"
_safe_save_csv(df_audit, audit_conflicts_path)

if "specific_name_clean" in df2.columns:
    dup_name = (df2.assign(_wfo=df2["wfo_id_final"].replace({"": np.nan}))
                   .dropna(subset=["_wfo"])
                   .groupby("specific_name_clean")["_wfo"].nunique()
                   .reset_index().query("_wfo > 1"))
    audit_name_multi_path = AUDIT_DIR / "audit_name_to_multiple_wfo.csv"
    _safe_save_csv(dup_name, audit_name_multi_path)
else:
    dup_name = None

wfo_counts = (df2.assign(_wfo=df2["wfo_id_final"].replace({"": np.nan}))
                .dropna(subset=["_wfo"])
                .groupby("_wfo")["specific_name_clean"].nunique()
                .reset_index(name="n_names").sort_values("n_names", ascending=False))
audit_wfo_multi_path  = AUDIT_DIR / "audit_wfo_to_multiple_names.csv"
_safe_save_csv(wfo_counts, audit_wfo_multi_path)

# ====== STEP 8: SUMMARY ======
total = len(df_bkf)
filled = total - len(unmatched)
src_counts = Counter(df2["wfo_source"])
conflicts = int(df2["any_conflict"].sum()) if "any_conflict" in df2.columns else None

print("\n=== COVERAGE & SOURCE ===")
print(f"Total BKF rows: {total}")
print(f"Filled (any source): {filled}  |  Coverage: {filled/total:.1%}")
for k in ["bkf_original","output7_exact","candidates_unique","unfilled"]:
    print(f"  {k:18s}: {src_counts.get(k,0)}")
print(f"Any conflicts flagged: {conflicts}")

summary = {
    "total_bkf_rows": total,
    "filled_rows": filled,
    "coverage_ratio": round(filled/total, 4),
    "source_breakdown": {str(k): int(v) for k,v in src_counts.items()},
    "any_conflicts": conflicts,
    "run_stamp": RUN_STAMP,
}
summary_json = SUMMARY_DIR / "coverage_summary.json"
with open(summary_json, "w", encoding="utf-8") as f:
    json.dump(summary, f, ensure_ascii=False, indent=2)
print(f"[OK] Saved summary: {summary_json}")

summary_csv = SUMMARY_DIR / "coverage_summary.csv"
pd.DataFrame([summary]).to_csv(summary_csv, index=False)
print(f"[OK] Saved summary CSV: {summary_csv}")

# ====== STEP 9: SNAPSHOT (copy to runs/<stamp>) ======
for p in [
    enriched_path, unmatched_path,
    audit_conflicts_path, audit_wfo_multi_path
]:
    if p.exists():
        dest = RUN_DIR / p.name
        shutil.copy2(p, dest)
        print(f"[SNAPSHOT] {p.name} -> {dest}")
if 'audit_name_multi_path' in locals():
    if audit_name_multi_path.exists():
        shutil.copy2(audit_name_multi_path, RUN_DIR / audit_name_multi_path.name)
        print(f"[SNAPSHOT] {audit_name_multi_path.name} -> {RUN_DIR / audit_name_multi_path.name}")
for p in [summary_json, summary_csv]:
    if p.exists():
        shutil.copy2(p, RUN_DIR / p.name)
        print(f"[SNAPSHOT] {p.name} -> {RUN_DIR / p.name}")



### 📝 อธิบายโค้ดส่วนสำคัญ
- **CONFIG:** กำหนดพาธ `ROOT_DIR`, โฟลเดอร์ input (`Raw data`) และ output (`Processed data`)  
- **HELPERS:** ฟังก์ชันทำความสะอาดชื่อและ WFO (`norm`, `wfo_clean`) และยูทิลิตี้ใช้งานทั่วไป  
- **STEP 0–3:** รวมข้อมูลจาก BKF + output(7) + candidates แล้วตัดสินใจ `wfo_id_final` ตามลำดับความสำคัญ  
- **STEP 4:** เติม `wfo_full_name` จาก mapping ในไฟล์ output(7)  
- **STEP 5:** จัดคอลัมน์ผลลัพธ์มาตรฐาน (`wfo_id`, `wfo_full_name`)  
- **STEP 6:** บันทึก **enriched** + **unmatched** ไปที่ `01_enriched/`  
- **STEP 7:** บันทึกไฟล์ **audit** (conflicts, name↔WFO, WFO↔names) ไปที่ `02_audit/`  
- **STEP 8:** สร้างไฟล์สรุป **summary (JSON/CSV)** ไปที่ `03_summary/`  
- **STEP 9:** ทำสำเนา snapshot ของไฟล์ผลลัพธ์ทั้งหมดไปไว้ที่ `runs/<timestamp>/`  
